# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/hexgrad/Kokoro-82M',
 '/openbmb/MiniCPM-o-2_6',
 '/deepseek-ai/DeepSeek-R1',
 '/MiniMaxAI/MiniMax-Text-01',
 '/microsoft/phi-4',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/FaceOnLive/Face-Search-Online',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/tran

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
hexgrad/Kokoro-82M
Updated
2 days ago
•
27.4k
•
2.07k
openbmb/MiniCPM-o-2_6
Updated
about 8 hours ago
•
18.3k
•
672
deepseek-ai/DeepSeek-R1
Updated
about 2 hours ago
•
441
MiniMaxAI/MiniMax-Text-01
Updated
3 days ago
•
2.7k
•
433
microsoft/phi-4
Updated
12 days ago
•
134k
•
1.47k
Browse 400k+ models
Spaces
Running
on
Zero
1.22k
❤️
Kokoro TTS
Now in 5 languages!
Running
on
Zero
3.15k
🏢
TRELLIS
Scalable and Versatile 3D Generation from images
Running
730
🔍😊
Search Your Face Onl

In [23]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nhexgrad/Kokoro-82M\nUpdated\n2 days ago\n•\n27.4k\n•\n2.07k\nopenbmb/MiniCPM-o-2_6\nUpdated\nabout 8 hours ago\n•\n18.3k\n•\n672\ndeepseek-ai/DeepSeek-R1\nUpdated\nabout 2 hours ago\n•\n441\nMiniMaxAI/MiniMax-Text-01\nUpdated\n3 days ago\n•\n2.7k\n•\n433\nmicrosoft/phi-4\nUpdated\n12 days ago\n•\n134k\n•\n1.47k\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.22k\n❤️\nKokoro TTS\nNow in 5 languages!\nRunning\non\nZero\n3.15k\n🏢\nT

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog', 'url': 'https://huggingface.com/blog'}, {'type': 'forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Hugging Face Brochure

### The AI Community Building the Future

**Hugging Face** is a collaborative platform where the machine learning community unites to create, discover, and innovate using models, datasets, and applications. With over **400,000 models** and **100,000 datasets**, Hugging Face empowers developers and researchers to accelerate their machine learning projects while sharing knowledge and resources.

---

### Our Vision 
At Hugging Face, we are committed to shaping the future of artificial intelligence through an open-source movement. We believe in the power of community collaboration and aim to provide the best tools and resources for everyone involved in machine learning.

---

### Company Culture
We foster an inclusive and innovative work environment that embraces creativity and collaboration. Our culture encourages personal and professional growth, valuing contributions from all team members. We are committed to work-life balance and a supportive atmosphere that prioritizes well-being and diversity.

---

### Customers 
Hugging Face is trusted by over **50,000 organizations**, including industry leaders like Amazon Web Services, Google, Microsoft, and Intel. These partnerships allow us to continue enhancing our platform and re-affirm our position as a pivotal player in the field of machine learning.

---

### Careers at Hugging Face
We are always on the lookout for passionate individuals who thrive in a collaborative and innovative environment. Whether you are a developer, researcher, or marketing expert, there’s a place for you at Hugging Face. Join us to contribute to cutting-edge AI technology while growing your career alongside talented peers.

**Open Positions Include:**
- Machine Learning Engineers
- Data Scientists
- Backend & Frontend Developers
- Community Managers

Explore current job openings on our [Careers Page](https://huggingface.co/jobs).

---

### Join Us
Be part of the **Hugging Face** community to accelerate your AI projects. Sign up today and start collaborating with a vibrant community that shares your passion for machine learning.

[Sign Up](https://huggingface.co/signup)

--- 

For more information, visit [Hugging Face](https://huggingface.co).
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face

## The AI Community Building the Future

Hugging Face is a leading platform where machine learning communities come together to collaborate on advanced models, datasets, and applications. We are proud to be at the forefront of AI development, supporting over **50,000 organizations** worldwide, including industry giants like Amazon, Google, Microsoft, and Intel.

---

## **Our Offerings**

### **Models and Datasets**
With access to **400,000+ models** and **100,000+ datasets**, Hugging Face supports a multitude of machine learning tasks—ranging from natural language processing to image generation. Key models trending this week include:
- Hexgrad/Kokoro-82M
- Openbmb/MiniCPM-o-2_6
- Microsoft/phi-4

### **Spaces**
Explore innovative applications in **Spaces** where AI solutions come to life:
- **Kokoro TTS**: Multilingual text-to-speech system.
- **TRELLIS**: A versatile 3D generation tool.

### **Compute and Enterprise Solutions**
Hugging Face provides **enterprise-grade solutions** to accelerate machine learning projects. With optimized inference endpoints and dedicated support, you can expect secure and scalable AI development. Pricing starts as low as **$0.60/hour for GPU**.

---

## **Our Culture**

At Hugging Face, we foster a collaborative and inclusive environment emphasizing innovation and community. Our mission is to democratize AI and make it accessible for all.

- **Open Source Commitment**: We believe in building AI tools with the community, developing state-of-the-art technologies such as Transformers, Diffusers, and Tokenizers that are used globally.
- **Community Engagement**: Be a part of our vibrant community through forums, Discord, and social platforms where you can learn, share, and grow together.

---

## **Careers and Opportunities**

Looking to join a forward-thinking company? Hugging Face is not only about cutting-edge technology but also about talented, passionate individuals who drive progress. We value creativity and empower our team members to build their careers in a supportive and dynamic environment.

- Discover open positions and be a part of shaping the future of AI.

---

## **Join Us**

Whether you're a researcher, developer, or a business leader, Hugging Face is the platform for you to explore, innovate, and collaborate on the future of AI. Sign up today to begin your journey!

[Visit our website](https://huggingface.co) and explore the possibilities!

---

**Contact Us**
For more inquiries, feel free to reach out via our support channels or connect with us on social media!

- **Twitter**: @huggingface
- **LinkedIn**: Hugging Face
- **Discord**: Join our community



In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face! 🤗

**Where AI dreams come true... with lots of cuddles!**

---

### Who are we? 🌍
Hugging Face is not just a platform; it's a thriving community of AI enthusiasts and professionals who believe in building the future—one friendly algorithm at a time! From models to datasets and everything in between, we’re the digital neighborhood where machine learning hangs out. 

---

### What do we do? 🛠️
- **Models**: As of this week, we’ve got **400k+ models** — that’s more models than there are flavors of ice cream (and we believe that’s a pretty big deal!). 
- **Datasets**: 100k+ datasets available, ready to be explored faster than you can say "Data is the new oil!" (Just don’t drill too deep; we’re environmentally friendly here!)
- **Spaces**: Think of this as the AI playground—except there are no sandcastles but plenty of innovative applications running on AI magic!

---

### Our Customers! 🧑‍💼
Over **50,000 organizations** are putting their trust in us (we promise, we're not just doing the awkward finger-gun thing). From giants like **Google, Amazon**, and **Microsoft**, to your favorite local startup; everyone is welcome! Join the club where even all your friends' models get along like best buddies at a summer camp!

---

### Company Culture! 🎉
- **Collaboration Central**: If you believe brainstorming is a sport, then Hugging Face is your Olympic stadium. Here, we practice **gentle brainwaves**, so don’t be surprised if you’re brainstorming with someone in shorts and flip-flops (we mean shorts on video calls, of course).
- **Innovation at Heart**: We live by the motto, “If it doesn’t break, it’s an old model!” (Just kidding, don't break things). We thrive on cutting-edge research and practice stellar teamwork!
- **Open Source Foundation**: We're building on trust and transparency—kinda like sharing popcorn at a movie (though less messy).

---

### Careers at Hugging Face! 📈
Think you can keep up with our vibe? Come work with us! Whether you’re an AI wizard or a data whisperer, we have roles that fit everyone. Here’s a sneak peek at potential jobs:
- **Model Maestros**: Create models that don't just work but also tell dad jokes.
- **Data Handlers**: Keep our datasets in shape while performing interpretive dances (optional).
- **Community Builders**: Gather the troops for chat, coffee, and code!

At Hugging Face, every day feels like the best day at the office. So sign up, log on, and let’s code like the wind! 🌬️💻

---

### Final Thoughts! 💡
Join us as we unravel the wonders of AI, one happy face at a time! Customers, investors, and recruits alike, we're excited for your next journey with us. Who knows? You might just end up with your own hug emoji. 

**Contact us!** 
Let’s cuddle with some codes! 
[www.huggingface.co](http://www.huggingface.co) | 📧 info@huggingface.co

---

*Hugging Face: Because even AI deserves a hug!*



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>